In [1]:
# # %% [code] {"execution":{"iopub.status.busy":"2024-11-08T15:30:33.966328Z","iopub.execute_input":"2024-11-08T15:30:33.966817Z","iopub.status.idle":"2024-11-08T15:30:33.975326Z","shell.execute_reply.started":"2024-11-08T15:30:33.966772Z","shell.execute_reply":"2024-11-08T15:30:33.973845Z"},"jupyter":{"outputs_hidden":false}}
# import pandas as pd
# import numpy as np
# from sklearn.feature_selection import VarianceThreshold, SelectKBest, f_classif
# from sklearn.model_selection import train_test_split
# from xgboost import XGBClassifier
# from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, roc_auc_score, roc_curve
# import matplotlib.pyplot as plt
# from sklearn.pipeline import Pipeline

# # %% [code] {"execution":{"iopub.status.busy":"2024-11-08T15:30:33.978000Z","iopub.execute_input":"2024-11-08T15:30:33.978637Z","iopub.status.idle":"2024-11-08T15:30:37.609880Z","shell.execute_reply.started":"2024-11-08T15:30:33.978576Z","shell.execute_reply":"2024-11-08T15:30:37.608747Z"},"jupyter":{"outputs_hidden":false}}
# # Load your dataset
# # Assuming your data is in a pandas dataframe called 'df'
# # 'target' is the column with the disease classification (1 or 0)
# df = pd.read_csv('/kaggle/input/imlcomp1/train_set.csv')
# X = df.drop('Y', axis=1)
# y = df['Y']

# # %% [code] {"execution":{"iopub.status.busy":"2024-11-08T15:30:37.611897Z","iopub.execute_input":"2024-11-08T15:30:37.612260Z","iopub.status.idle":"2024-11-08T15:30:37.679913Z","shell.execute_reply.started":"2024-11-08T15:30:37.612222Z","shell.execute_reply":"2024-11-08T15:30:37.678727Z"},"jupyter":{"outputs_hidden":false}}
# X

# # %% [code] {"execution":{"iopub.status.busy":"2024-11-08T15:30:37.681433Z","iopub.execute_input":"2024-11-08T15:30:37.681937Z","iopub.status.idle":"2024-11-08T15:30:37.692354Z","shell.execute_reply.started":"2024-11-08T15:30:37.681875Z","shell.execute_reply":"2024-11-08T15:30:37.691033Z"},"jupyter":{"outputs_hidden":false}}
# y

# # %% [code] {"execution":{"iopub.status.busy":"2024-11-08T15:30:37.695085Z","iopub.execute_input":"2024-11-08T15:30:37.695503Z","iopub.status.idle":"2024-11-08T15:30:38.226956Z","shell.execute_reply.started":"2024-11-08T15:30:37.695459Z","shell.execute_reply":"2024-11-08T15:30:38.225877Z"},"jupyter":{"outputs_hidden":false}}
# dfX_filled = X.fillna(X.mean())
# dfY_filled = y.fillna(y.mean())
# df_filled = df.fillna(df.mean())

# # %% [code] {"execution":{"iopub.status.busy":"2024-11-08T15:30:38.228904Z","iopub.execute_input":"2024-11-08T15:30:38.229379Z","iopub.status.idle":"2024-11-08T15:30:38.238308Z","shell.execute_reply.started":"2024-11-08T15:30:38.229335Z","shell.execute_reply":"2024-11-08T15:30:38.237208Z"},"jupyter":{"outputs_hidden":false}}
# # Calculate the number of true positives and true negatives
# true_positive_count = (df_filled['Y'] == 1).sum()  # Count of true positives
# true_negative_count = (df_filled['Y'] == 0).sum()  # Count of true negatives

# # Display the results
# print(f"True Positive Count: {true_positive_count}")
# print(f"True Negative Count: {true_negative_count}")

# # %% [code] {"execution":{"iopub.status.busy":"2024-11-08T15:30:38.239839Z","iopub.execute_input":"2024-11-08T15:30:38.240190Z","iopub.status.idle":"2024-11-08T15:30:39.801446Z","shell.execute_reply.started":"2024-11-08T15:30:38.240152Z","shell.execute_reply":"2024-11-08T15:30:39.800270Z"},"jupyter":{"outputs_hidden":false}}
# # Separate the positive and negative instances
# df_positive = df_filled[df_filled['Y'] == 1]
# df_negative = df_filled[df_filled['Y'] == 0]

# # Calculate how many more positive instances are needed
# needed_positive_count = true_negative_count - true_positive_count

# # Upscale positive instances by sampling with replacement
# if needed_positive_count > 0:
#     df_positive_upscaled = df_positive.sample(n=needed_positive_count, replace=True, random_state=42)
# else:
#     df_positive_upscaled = pd.DataFrame()  # No upscaling needed if positive count is already equal to or greater

# # Combine the original positive instances with the upscaled ones
# df_balanced = pd.concat([df_positive, df_positive_upscaled, df_negative])

# # Shuffle the resulting DataFrame (optional)
# df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

# # Display the balanced DataFrame
# print("Balanced DataFrame:")
# print(df_balanced)
# print(f"New Positive Count: {(df_balanced['Y'] == 1).sum()}")
# print(f"New Negative Count: {(df_balanced['Y'] == 0).sum()}")

# # %% [code] {"execution":{"iopub.status.busy":"2024-11-08T15:30:39.803078Z","iopub.execute_input":"2024-11-08T15:30:39.803480Z","iopub.status.idle":"2024-11-08T15:30:39.810115Z","shell.execute_reply.started":"2024-11-08T15:30:39.803426Z","shell.execute_reply":"2024-11-08T15:30:39.808828Z"},"jupyter":{"outputs_hidden":false}}
# df = df_balanced

# # %% [code] {"execution":{"iopub.status.busy":"2024-11-08T15:30:39.811702Z","iopub.execute_input":"2024-11-08T15:30:39.812130Z","iopub.status.idle":"2024-11-08T15:30:39.917753Z","shell.execute_reply.started":"2024-11-08T15:30:39.812086Z","shell.execute_reply":"2024-11-08T15:30:39.916482Z"},"jupyter":{"outputs_hidden":false}}
# X = df.drop('Y', axis=1)
# y = df['Y']

# # %% [code] {"execution":{"iopub.status.busy":"2024-11-08T15:30:39.919955Z","iopub.execute_input":"2024-11-08T15:30:39.920367Z","iopub.status.idle":"2024-11-08T15:30:40.358803Z","shell.execute_reply.started":"2024-11-08T15:30:39.920324Z","shell.execute_reply":"2024-11-08T15:30:40.357534Z"},"jupyter":{"outputs_hidden":false}}
# # Split the data
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# # %% [code] {"execution":{"iopub.status.busy":"2024-11-08T15:30:40.363147Z","iopub.execute_input":"2024-11-08T15:30:40.363582Z","iopub.status.idle":"2024-11-08T15:30:40.370549Z","shell.execute_reply.started":"2024-11-08T15:30:40.363517Z","shell.execute_reply":"2024-11-08T15:30:40.369251Z"},"jupyter":{"outputs_hidden":false}}
# pipeline = Pipeline([
#     ('var_thresh', VarianceThreshold(threshold=0.1)),  # Remove features with low variance
#     ('k_best', SelectKBest(score_func=f_classif, k=20)),  # Select top 20 features
#     ('xgb', XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42))
# ])

# # %% [code] {"execution":{"iopub.status.busy":"2024-11-08T15:30:40.372494Z","iopub.execute_input":"2024-11-08T15:30:40.372981Z","iopub.status.idle":"2024-11-08T15:30:44.543882Z","shell.execute_reply.started":"2024-11-08T15:30:40.372934Z","shell.execute_reply":"2024-11-08T15:30:44.542778Z"},"jupyter":{"outputs_hidden":false}}
# pipeline.fit(X_train, y_train)

# # %% [code] {"execution":{"iopub.status.busy":"2024-11-08T15:30:44.545652Z","iopub.execute_input":"2024-11-08T15:30:44.546626Z","iopub.status.idle":"2024-11-08T15:30:44.939005Z","shell.execute_reply.started":"2024-11-08T15:30:44.546571Z","shell.execute_reply":"2024-11-08T15:30:44.937978Z"},"jupyter":{"outputs_hidden":false}}
# # Make predictions
# y_pred = pipeline.predict(X_test)
# y_pred_proba = pipeline.predict_proba(X_test)[:, 1]  # Probability estimates for ROC-AUC

# # %% [code] {"execution":{"iopub.status.busy":"2024-11-08T15:30:44.940656Z","iopub.execute_input":"2024-11-08T15:30:44.941416Z","iopub.status.idle":"2024-11-08T15:30:45.616231Z","shell.execute_reply.started":"2024-11-08T15:30:44.941372Z","shell.execute_reply":"2024-11-08T15:30:45.615045Z"},"jupyter":{"outputs_hidden":false}}
# # Evaluate the model
# print("Accuracy:", accuracy_score(y_test, y_pred))
# print("Confusion Matrix:")
# print(confusion_matrix(y_test, y_pred))
# print(classification_report(y_test, y_pred))

# # Calculate AUC of ROC
# roc_auc = roc_auc_score(y_test, y_pred_proba)
# print("AUC of ROC:", roc_auc)

# # Plot ROC curve
# fpr, tpr, _ = roc_curve(y_test, y_pred_proba)
# plt.figure()
# plt.plot(fpr, tpr, color='blue', label='ROC curve (AUC = %0.2f)' % roc_auc)
# plt.plot([0, 1], [0, 1], color='grey', linestyle='--')
# plt.xlabel('False Positive Rate')
# plt.ylabel('True Positive Rate')
# plt.title('Receiver Operating Characteristic (ROC) Curve')
# plt.legend(loc="lower right")
# plt.show()

# # %% [code] {"execution":{"iopub.status.busy":"2024-11-08T15:30:45.617904Z","iopub.execute_input":"2024-11-08T15:30:45.618385Z","iopub.status.idle":"2024-11-08T15:30:47.181973Z","shell.execute_reply.started":"2024-11-08T15:30:45.618333Z","shell.execute_reply":"2024-11-08T15:30:47.180861Z"},"jupyter":{"outputs_hidden":false}}
# df_test = pd.read_csv('/kaggle/input/imlcomp1/test_set.csv')

# # %% [code] {"execution":{"iopub.status.busy":"2024-11-08T15:30:47.183778Z","iopub.execute_input":"2024-11-08T15:30:47.184190Z","iopub.status.idle":"2024-11-08T15:30:47.291281Z","shell.execute_reply.started":"2024-11-08T15:30:47.184147Z","shell.execute_reply":"2024-11-08T15:30:47.290096Z"},"jupyter":{"outputs_hidden":false}}
# df_test_filled = df_test.fillna(df_test.mean())

# # %% [code] {"execution":{"iopub.status.busy":"2024-11-08T15:30:47.292708Z","iopub.execute_input":"2024-11-08T15:30:47.293096Z","iopub.status.idle":"2024-11-08T15:30:47.690109Z","shell.execute_reply.started":"2024-11-08T15:30:47.293055Z","shell.execute_reply":"2024-11-08T15:30:47.689051Z"},"jupyter":{"outputs_hidden":false}}
# # Make predictions
# y_pred = pipeline.predict(df_test_filled)
# y_pred_proba = pipeline.predict_proba(df_test_filled)[:, 1]  # Probability estimates for ROC-AUC

# # %% [code] {"execution":{"iopub.status.busy":"2024-11-08T15:30:47.691865Z","iopub.execute_input":"2024-11-08T15:30:47.692403Z","iopub.status.idle":"2024-11-08T15:30:47.891270Z","shell.execute_reply.started":"2024-11-08T15:30:47.692339Z","shell.execute_reply":"2024-11-08T15:30:47.889906Z"},"jupyter":{"outputs_hidden":false}}
# # Save the test predictions (if needed)
# df_test_filled['Y'] = y_pred
# df_test_filled[['RecordId', 'Y']].to_csv('test_predictions.csv', index=False)

# print("Predictions made on the test set and saved to 'test_predictions.csv'.")

# # %% [code] {"execution":{"iopub.status.busy":"2024-11-08T15:30:47.892799Z","iopub.execute_input":"2024-11-08T15:30:47.893274Z","iopub.status.idle":"2024-11-08T15:30:47.899984Z","shell.execute_reply.started":"2024-11-08T15:30:47.893221Z","shell.execute_reply":"2024-11-08T15:30:47.898703Z"},"jupyter":{"outputs_hidden":false}}
# print("done")

In [2]:
import pandas as p
print("Training set")
train_df = p.read_csv('/kaggle/input/imlcomp1/train_set.csv')
test_df = p.read_csv('/kaggle/input/imlcomp1/test_set.csv')
print(train_df.head())
print("test set")
print(test_df.head())
from sklearn.impute import KNNImputer

Training set
   RecordId         X2         X3  X4  X5  X6          X7  X8   X9  X10  ...  \
0         1  87.000000  34.118411   0   2   0  165.100000   1  829    2  ...   
1         2  82.372284  31.573280   0   0   1  162.983897   1  724    0  ...   
2         3  50.000000  27.771653   0   0   1  165.100000   1  895    2  ...   
3         4  66.236109  26.515922   0   0   1  167.009549   1  637    0  ...   
4         5  81.303299  20.843691   0   0   1  158.165419   0  564    0  ...   

        X70  X71  X72  X73  X74  X75  X76  X77  X78  Y  
0  0.040000  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0  
1  0.033431  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0  
2  0.010000  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0  
3  0.039363  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0  
4  0.069242  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0  

[5 rows x 79 columns]
test set
   RecordId         X2         X3  X4  X5  X6          X7  X8   X9  X10  ...  \
0    300001  79.000000  17.122318   0   0   1  170.2

In [3]:
imputer = KNNImputer(n_neighbors=25)
train_df.loc[:, train_df.columns != 'Y'] = imputer.fit_transform(train_df.loc[:, train_df.columns != 'Y'])

In [4]:
dropped_columns = []

In [5]:
import pandas as pd
import numpy as np  # Explicitly import numpy

In [6]:
# Calculate the correlation matrix
correlation_matrix = train_df.corr().abs()

In [7]:
# Set threshold for removing correlated features before it was 0.85
correlation_threshold = 0.8

In [8]:
# Select upper triangle of the correlation matrix
upper_triangle = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool))

In [9]:
# Identify features to drop based on the correlation threshold
to_drop = [column for column in upper_triangle.columns if any(upper_triangle[column] > correlation_threshold)]

In [10]:
# Store the names of dropped columns
dropped_columns = to_drop.copy()

In [11]:
# Drop the highly correlated features from the training set
train_df_reduced = train_df.drop(columns=to_drop)

In [12]:
# Redefine Features and Target with reduced features
Features_reduced = train_df_reduced.drop(columns=['RecordId', 'Y'])
Target = train_df_reduced['Y']

In [13]:
# Print the reduced feature set
print(f"Features reduced from {train_df.shape[1]} to {train_df_reduced.shape[1]}")
print(Features_reduced.head())
print(dropped_columns)

Features reduced from 79 to 58
          X2         X3  X4  X5  X6          X7  X8   X9  X10  X11  ...  \
0  87.000000  34.118411   0   2   0  165.100000   1  829    2    7  ...   
1  82.372284  31.573280   0   0   1  162.983897   1  724    0    4  ...   
2  50.000000  27.771653   0   0   1  165.100000   1  895    2    7  ...   
3  66.236109  26.515922   0   0   1  167.009549   1  637    0    7  ...   
4  81.303299  20.843691   0   0   1  158.165419   0  564    0    5  ...   

        X68       X69  X71  X72  X73  X74  X75  X76  X77  X78  
0  4.200000  0.110000  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
1  3.718976  0.100292  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
2  3.800000  0.020000  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
3  4.285677  0.108249  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
4  3.769194  0.164645  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  

[5 rows x 56 columns]
['X16', 'X29', 'X30', 'X31', 'X33', 'X34', 'X35', 'X36', 'X43', 'X44', 'X49', 'X50', 'X52', 'X53', 'X54', 'X55'

In [14]:
# Scaling
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()
Features = scaler.fit_transform(Features_reduced)

In [15]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
import xgboost as xgb
import random

In [16]:
# Assuming Features_reduced_cleaned and Target are already defined
# Step 1: Perform a 70/30 train-test split
train_f, val_f, train_t, val_t = train_test_split(Features_reduced, Target, test_size=0.30, random_state=42)

In [17]:
# Step 2: Set up data for XGBoost
train_data = xgb.DMatrix(train_f, label=train_t)
val_data = xgb.DMatrix(val_f, label=val_t)

In [18]:
base_params = {
    'objective': 'binary:logistic',
    'sampling_method': 'uniform',           # Uniform sampling for CPU-based 'hist' method
    'eval_metric': 'auc',                   # AUC metric to assess model performance
    'max_depth': 10,                        # Reduced depth for better generalization
    'eta': 0.1,                             # Lower learning rate for smaller, more refined steps
    'num_parallel_tree': 5,                 # Reduced to simplify the ensemble and control overfitting
    'subsample': 0.8,                       # Slightly increased for diverse sampling
    'colsample_bytree': 0.5,                # Increase to include more features per tree
    'min_child_weight': 3,                  # Increased for more conservative splits
    'gamma': 0.25,                          # Higher threshold to reduce overfitting on noisy splits
    'alpha': 3,                             # Stronger L1 regularization to control model complexity
    'lambda': 3,                            # Stronger L2 regularization to control model complexity
    'max_delta_step': 5,                    # Further increasing for imbalance handling
    'scale_pos_weight': 1.5,                # Adjustment to handle class imbalance
    'tree_method': 'hist',                  # Using 'hist' method for large datasets (CPU-optimized)
    'grow_policy': 'lossguide',             # Guides tree growth based on loss reduction for better accuracy
    'verbosity': 2                          # Verbose mode to monitor training progress
}

In [19]:
# Step 3: Train multiple XGBoost models with different seeds and other varying parameters

num_rounds = 350

In [20]:
# Train the model
model = xgb.train(base_params, train_data, num_boost_round=num_rounds, verbose_eval=True)

In [21]:
# Predict on validation set with all iterations (0 to num_rounds)
pred_prob = model.predict(val_data, iteration_range=(0, num_rounds))
val_predictions = [1 if prob > 0.5 else 0 for prob in pred_prob]

In [22]:
# Step 5: Calculate accuracy
accuracy = accuracy_score(val_t, val_predictions)
print("Validation Accuracy:", accuracy)

Validation Accuracy: 0.9976570012324444


In [23]:
# Predict on test set
# Step 6: Prepare the test data and drop unnecessary columns
test_df_imputed = imputer.transform(test_df)
test_features = pd.DataFrame(test_df_imputed, columns=test_df.columns)
test_features = test_features.drop(columns=['RecordId'] + dropped_columns)

In [24]:
# Step 7: Convert the test features to DMatrix format
test_data = xgb.DMatrix(test_features)

In [25]:
# Step 8: Make predictions on the test set with all iterations
test_pred_prob = model.predict(test_data, iteration_range=(0, num_rounds))

In [26]:
# Step 9: Create the submission DataFrame
kaggle_submission = pd.DataFrame({'RecordId': test_df['RecordId'], 'Y': test_pred_prob})
kaggle_submission.to_csv('day1_64th_submission.csv', index=False)
print('Submission file created successfully.')

Submission file created successfully.


In [27]:
# import pandas as p
# print("Training set")
# train_df = p.read_csv('train_set.csv')
# test_df = p.read_csv('test_set.csv')
# print(train_df.head())
# print("test set")
# print(test_df.head())
# from sklearn.impute import SimpleImputer

# imputer = SimpleImputer(strategy='median')  # or 'median', 'most_frequent', 'constant'
# train_df.loc[:, train_df.columns != 'Y'] = imputer.fit_transform(train_df.loc[:, train_df.columns != 'Y'])

# dropped_columns = []
# # Block 3: Calculating Correlation and Reducing Features
# import pandas as pd
# import numpy as np  # Explicitly import numpy

# # Calculate the correlation matrix
# correlation_matrix = train_df.corr().abs()

# # Set threshold for removing correlated features before it was 0.85
# correlation_threshold = 0.75

# # Select upper triangle of the correlation matrix
# upper_triangle = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool))

# # Identify features to drop based on the correlation threshold
# to_drop = [column for column in upper_triangle.columns if any(upper_triangle[column] > correlation_threshold)]

# # Store the names of dropped columns
# dropped_columns = to_drop.copy()

# # Drop the highly correlated features from the training set
# train_df_reduced = train_df.drop(columns=to_drop)

# # Redefine Features and Target with reduced features
# Features_reduced = train_df_reduced.drop(columns=['RecordId', 'Y'])
# Target = train_df_reduced['Y']

# # Print the reduced feature set
# print(f"Features reduced from {train_df.shape[1]} to {train_df_reduced.shape[1]}")
# print(Features_reduced.head())
# print(dropped_columns)
# #Scaling
# from sklearn.preprocessing import StandardScaler

# scaler = StandardScaler()
# Features = scaler.fit_transform(Features_reduced)
# import pandas as pd
# from sklearn.ensemble import RandomForestClassifier
# import xgboost as xgb
# from sklearn.ensemble import ExtraTreesClassifier
# from sklearn.metrics import accuracy_score
# from sklearn.model_selection import train_test_split
# # Assuming Features_reduced_cleaned and target_values are already defined
# # Step 1: Perform a 70/30 train-test split
# train_f, val_f, train_t, val_t = train_test_split(Features_reduced, Target, test_size=0.30, random_state=42)
# #new 
# #rf_model = RandomForestClassifier(criterion='gini',random_state=50, n_estimators=20000, max_depth=9, max_features = 7,n_jobs=-1,class_weight='balanced',verbose=2)
# # Step 3: Train the model on the training data
# #rf_model.fit(train_f, train_t)
# # Step 4: Predict on the validation set
# #val_predictions = rf_model.predict(val_f)
# ## trying xgboost
# train_data = xgb.DMatrix(train_f,label=train_t)
# test_data = xgb.DMatrix(val_f,label=val_t)
# params = {
#     'sampling_method':'uniform',
#     'num_parallel_tree':10,
#     'eval_metric' : 'auc',
#     'objective': 'binary:logistic',
#     'max_depth':12,
#     'eta': 0.05,
#     'subsample': 0.8,
#     'colsample_bytree': 0.3,
#     'seed': 42, 
#     'nthread': -1,            # Use all CPU cores
#     'tree_method': 'hist', # Use GPU for acceleration
#     'verbosity': 2            # Control verbosity (0, 1, or 2)
# }
# num_rounds = 2000
# model = xgb.train(params, train_data, num_boost_round=num_rounds,verbose_eval=2)
# pred_prob = model.predict(test_data)
# val_predictions  = [1 if prob > 0.5 else 0 for prob in pred_prob]
# # Step 5: Calculate accuracy
# accuracy = accuracy_score(val_t, val_predictions)
# # Step 6: Output the validation accuracy
# print("Validation Accuracy:", accuracy)
# #new portion below
# # Step 1: Impute the test data using the same strategy as training
# test_df_imputed = imputer.transform(test_df)
# # Step 2: Convert back to DataFrame and drop columns based on training
# test_features = pd.DataFrame(test_df_imputed, columns=test_df.columns)
# test_features = test_features.drop(columns=['RecordId'] +dropped_columns)
# #new portion above
# # Step 7: Prepare the test features by dropping the same columns as in training
# #test_features = test_df.drop(columns=['RecordId'] + dropped_columns)
# # Step 8: Make predictions on the test set
# # Step 6: Convert the test features to DMatrix format
# test_data = xgb.DMatrix(test_features)
# # Step 7: Make predictions on the test set
# test_pred_prob = model.predict(test_data)
# # For binary classification, we could use probabilities if needed. 
# # Here we get the predicted probabilities for class 1
# test_pred = test_pred_prob  # probabilities for class 1, as requested for submission
# # Step 8: Create the submission DataFrame
# kaggle_submission = pd.DataFrame({'RecordId': test_df['RecordId'], 'Y': test_pred})
# # Step 9: Save the submission file
# kaggle_submission.to_csv('day1_25th_submission.csv', index=False)
# print('Submission file created successfully.')